In [1]:
import requests
import json
import io
import zipfile
from bs4 import BeautifulSoup as bs
import time
import mysql.connector as server

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'}
no_incluir = ["Table", "SharesAxis", "Member", "LineItems", "Explanatory"]
lista_links_emisoras = []
ya_descargadas = []

db = server.connect(host="192.168.100.200", user="mensajero", passwd="password", db="db_trading")
query = db.cursor()
query.execute("SELECT DISTINCT(emisora), razon_social FROM emisoras")
lista_emisoras = query.fetchall()
query.execute("SELECT CONCAT(emisora, '_' ,periodo) FROM fundamentales")
ya_descargadas_bruto = query.fetchall()
db.close()

for cada in ya_descargadas_bruto:
    ya_descargadas.append(cada[0])

"""Descarga de links de XBRL"""


def ListadoLinks():
    url = "https://www.bmv.com.mx/es/emisoras/archivos-estadar-xbrl"
    respuesta = requests.get(url, headers=headers, timeout=60)
    if "Sitio Grupo BMV no disponible" in respuesta.text:
        raise Exception("El servidor de la BMV se encuentra en mantenimiento")

    sopadecoditos = bs(respuesta.content, "html.parser")
    tabla_zips = sopadecoditos.find("tbody").findAll("tr")
    d_links = {}

    for row in tabla_zips:
        texto_tabla = row.find_all("td")
        emisora = texto_tabla[0].get_text()
        link_tabla = row.find_all("a")
        link = str(link_tabla[0].get("href")).replace("/docs-pub/ifrsxbrl/../visor/visorXbrl.html?docins=../", "https://www.bmv.com.mx/docs-pub/")
        periodo = str(link_tabla[0].get_text()).replace("\n", "")
        lista_hija = []
        lista_padre = []

        if "anexon" not in link:
            lista_hija.append(link)
            lista_hija.append(periodo)
            lista_padre.append(lista_hija)

            if emisora in d_links:
                d_links[emisora].extend(lista_padre)
            else:
                d_links[emisora] = lista_padre
    return d_links


d_links = ListadoLinks()

for emisora_links in d_links:
    lista_links_emisoras.append(emisora_links)


def maquila_eeff(uuid_eeff, codigo_eeff, XBRL_Taxonomia, XBRL_HechosPorId, id_efectivo, uuid_inicio_efectivo, uuid_cierre_efectivo):
    eeff_del_periodo = {}
    eeff_del_periodo_ifrs = {}

    if uuid_eeff:
        for rol in XBRL_Taxonomia["RolesPresentacion"]:
            if codigo_eeff in rol["Nombre"]:
                catalogo_cuentas = str(rol["Estructuras"][0])
                break

        catalogo_cuentas = catalogo_cuentas.replace("[", "").replace("]", "").replace("{", "").replace("}", "").replace("'", "").replace(" ", "").split(",")

        for cuenta_ifrs in catalogo_cuentas:
            if "IdConcepto:" in cuenta_ifrs and not any(nope in cuenta_ifrs for nope in no_incluir):
                cuenta_ifrs = cuenta_ifrs.replace("IdConcepto:", "").replace("SubEstructuras:", "")
                eeff_del_periodo_ifrs[cuenta_ifrs] = ""
                cuenta = cuenta_ifrs.replace("ifrs-full_", "").replace("ifrs_mx-cor_20141205_", "")
                eeff_del_periodo[cuenta] = [XBRL_Taxonomia["ConceptosPorId"][cuenta_ifrs]["Etiquetas"]["es"]["http://www.xbrl.org/2003/role/label"]["Valor"]
                                                .replace(" [sinopsis]", "").replace("[sinopsis]", "")]

        for hecho, contenido in XBRL_HechosPorId.items():
            if contenido["IdContexto"] == uuid_eeff and contenido["IdConcepto"] in eeff_del_periodo_ifrs.keys():
                eeff_del_periodo[contenido["IdConcepto"].replace("ifrs-full_", "").replace("ifrs_mx-cor_20141205_", "")].append(contenido["ValorNumerico"])

        if codigo_eeff == "[520000]":
            eeff_del_periodo["CashAndCashEquivalents"] = [XBRL_Taxonomia["ConceptosPorId"][id_efectivo]["Etiquetas"]["es"]
                                                          ["http://www.xbrl.org/2003/role/periodStartLabel"]["Valor"]]
            eeff_del_periodo["CashAndCashEquivalents_Ending"] = [XBRL_Taxonomia["ConceptosPorId"][id_efectivo]["Etiquetas"]["es"]
                                                                 ["http://www.xbrl.org/2003/role/periodEndLabel"]["Valor"]]

            for key, value in XBRL_HechosPorId.items():
                if value["IdContexto"] == uuid_inicio_efectivo[2] and value["IdConcepto"] == id_efectivo:
                    eeff_del_periodo["CashAndCashEquivalents"] = [XBRL_Taxonomia["ConceptosPorId"][id_efectivo]["Etiquetas"]["es"]
                                                                  ["http://www.xbrl.org/2003/role/periodStartLabel"]["Valor"], value["ValorNumerico"]]

                if value["IdContexto"] == uuid_cierre_efectivo[2] and value["IdConcepto"] == id_efectivo:
                    eeff_del_periodo["CashAndCashEquivalents_Ending"] = [XBRL_Taxonomia["ConceptosPorId"][id_efectivo]["Etiquetas"]["es"]
                                                                         ["http://www.xbrl.org/2003/role/periodEndLabel"]["Valor"], value["ValorNumerico"]]

        return eeff_del_periodo


for emisora in lista_emisoras:
    if emisora[0] in lista_links_emisoras:
        for link in d_links[emisora[0]]:
            url = link[0]
            periodo_texto = link[1]
            if periodo_texto.find("para el") > 0:
                periodo_texto = periodo_texto.replace(periodo_texto[periodo_texto.find("para el"):len(periodo_texto)], "")
            periodo_texto = periodo_texto.replace("Descargar Información Del Trimestre ", "")
            periodo_texto = periodo_texto.replace("Del Año ", "T_")
            periodo_texto = periodo_texto.replace(" ", "")

            if f"{emisora[0]}_{periodo_texto}" not in ya_descargadas:
                print(f"Procesando {emisora[0]}_{periodo_texto}")

                def tres_estados_financieros():
                    l_contextos_unicos = []
                    estados_financieros = {}
                    uuids_maestros = {}

                    while True:
                        if str(url[-3:]).lower() == "zip":
                            try:
                                respuesta = requests.get(url, stream=True)
                            except requests.exceptions.ConnectionError:
                                print("El servidor rechaza solicitud, reintentando en 2 minutos.")
                                time.sleep(60)
                                continue
                            archivo_zip = zipfile.ZipFile(io.BytesIO(respuesta.content), 'r')
                            XBRL = json.loads(archivo_zip.read(archivo_zip.namelist()[0]))
                        elif str(url[-4:]).lower() == "json":
                            try:
                                respuesta = requests.get(url)
                            except requests.exceptions.ConnectionError:
                                print("El servidor rechaza solicitud, reintentando en 2 minutos.")
                                time.sleep(60)
                                continue
                            soupadecoditos = bs(respuesta.content, "html.parser")
                            XBRL = json.loads(soupadecoditos.prettify())
                        else:
                            raise Exception(f"La URL '{url}' no termina en .zip o .json, verificar lista de links")
                        break

                    id_periodo_reporte = XBRL["HechosPorIdConcepto"]["ifrs-full_DateOfEndOfReportingPeriod2013"][0]
                    fecha_cierre_original = XBRL["HechosPorId"][id_periodo_reporte]["Valor"]
                    fecha_cierre = fecha_cierre_original

                    if int(fecha_cierre_original[0:4]) < 2010:
                        id_moneda = "ifrs_DescriptionOfPresentationCurrency"
                        id_clave_pizarra = "mx-ifrs-ics_ClaveCotizacion"
                        id_nombre_emisora = "ifrs_NameOfReportingEntityOrOtherMeansOfIdentification"
                    else:
                        id_moneda = "ifrs-full_DescriptionOfPresentationCurrency"
                        id_clave_pizarra = "ifrs_mx-cor_20141205_ClaveDeCotizacionBloqueDeTexto"
                        #id_clave_pizarra = "mx_trac_Ticker"
                        id_nombre_emisora = "ifrs-full_NameOfReportingEntityOrOtherMeansOfIdentification"

                    moneda = str(XBRL["HechosPorId"][XBRL["HechosPorIdConcepto"][id_moneda][0]]["Valor"]).upper()
                    clave_pizarra = str(XBRL["HechosPorId"][XBRL["HechosPorIdConcepto"][id_clave_pizarra][0]]["Valor"]).upper()
                    nombre_emisora = str(XBRL["HechosPorId"][XBRL["HechosPorIdConcepto"][id_nombre_emisora][0]]["Valor"]).upper()

                    for concepto in XBRL["ContextosPorId"]:
                        if not XBRL["ContextosPorId"][concepto]["ContieneInformacionDimensional"]:
                            l_contextos_unicos.append(XBRL["ContextosPorId"][concepto]["Id"])

                    """Tranformacion de XBRL a JSON"""
                    for ronda in range(2):
                        if ronda == 1:
                            fecha_cierre = str(int(fecha_cierre[0:4]) - 1) + fecha_cierre[4:10]

                        anio_actual = str(fecha_cierre[0:4])
                        fecha_inicio = anio_actual + "-01-01"
                        fecha_inicio_y_cierre = fecha_inicio + "_" + fecha_cierre
                        titulo_resultado_flujos = f"Del {fecha_inicio} Al {fecha_cierre}"
                        fecha_cierre_anterior = str(int(anio_actual) - 1) + "-12-31"
                        fecha_cierre_actual = fecha_cierre

                        if fecha_cierre[5:7] == "12":
                            fecha_trimestre = f"{fecha_cierre[0:4]}-10-01_{fecha_cierre}"
                        if fecha_cierre[5:7] == "09":
                            fecha_trimestre = f"{fecha_cierre[0:4]}-07-01_{fecha_cierre}"
                        if fecha_cierre[5:7] == "06":
                            fecha_trimestre = f"{fecha_cierre[0:4]}-04-01_{fecha_cierre}"
                        if fecha_cierre[5:7] == "03":
                            fecha_trimestre = f"{fecha_cierre[0:4]}-01-01_{fecha_cierre}"

                        if ronda == 1:
                            fecha_cierre = str(fecha_cierre[0:4]) + "-12-31"

                        if int(fecha_cierre[0:4]) < 2010:
                            id_efectivo = "ifrs_CashAndCashEquivalents"
                        else:
                            id_efectivo = "ifrs-full_CashAndCashEquivalents"

                        for unicontexto in l_contextos_unicos:
                            try:
                                if unicontexto in XBRL["ContextosPorFecha"][fecha_cierre]: uuids_maestros["uuid_posicion"] = ["posicion", "[210000]", unicontexto, fecha_cierre]
                            except KeyError:
                                uuids_maestros["uuid_posicion"] = ["posicion", "[210000]", False, fecha_cierre]
                            try:
                                if unicontexto in XBRL["ContextosPorFecha"][fecha_inicio_y_cierre]:
                                    uuids_maestros["uuid_resultado"] = ["resultado_acumulado", "[310000]", unicontexto, fecha_inicio_y_cierre]
                                    uuids_maestros["uuid_flujos"] = ["flujos", "[520000]", unicontexto, fecha_inicio_y_cierre]
                            except KeyError:
                                uuids_maestros["uuid_resultado"] = ["resultado_acumulado", "[310000]", False, fecha_inicio_y_cierre]
                                uuids_maestros["uuid_flujos"] = ["flujos", "[520000]", False, fecha_inicio_y_cierre]
                            try:
                                if unicontexto in XBRL["ContextosPorFecha"][fecha_trimestre]: uuids_maestros["uuid_trimestre"] = ["resultado_trimestre", "[310000]", unicontexto, fecha_trimestre]
                            except KeyError:
                                uuids_maestros["uuid_trimestre"] = ["resultado_trimestre", "[310000]", False, False]
                            try:
                                if unicontexto in XBRL["ContextosPorFecha"][fecha_cierre_anterior]: uuids_maestros["uuid_inicio_efectivo"] = [False, False, unicontexto, False]
                            except KeyError:
                                uuids_maestros["uuid_inicio_efectivo"] = [False, False, False, False]
                            try:
                                if unicontexto in XBRL["ContextosPorFecha"][fecha_cierre_actual]: uuids_maestros["uuid_cierre_efectivo"] = [False, False, unicontexto, False]
                            except KeyError:
                                uuids_maestros["uuid_cierre_efectivo"] = [False, False, False, False]

                        for nombre_uuid, llaves in uuids_maestros.items():
                            if llaves[0] in ("posicion", "resultado_acumulado", "resultado_trimestre", "flujos") and llaves[2]:
                                if llaves[0] not in estados_financieros.keys():
                                    estados_financieros[llaves[0]] = {llaves[3]: maquila_eeff(llaves[2], llaves[1], XBRL["Taxonomia"], XBRL["HechosPorId"], id_efectivo, uuids_maestros["uuid_inicio_efectivo"], uuids_maestros["uuid_cierre_efectivo"])}
                                else:
                                    estados_financieros[llaves[0]] = {list(estados_financieros[llaves[0]].keys())[0]:
                                                                      estados_financieros[llaves[0]][list(estados_financieros[llaves[0]].keys())[0]], llaves[3]:
                                                                      maquila_eeff(llaves[2], llaves[1], XBRL["Taxonomia"], XBRL["HechosPorId"], id_efectivo, uuids_maestros["uuid_inicio_efectivo"], uuids_maestros["uuid_cierre_efectivo"])}
                    return estados_financieros
                
                
                carga_eeff = tres_estados_financieros()
                carga_eeff = json.dumps(carga_eeff, ensure_ascii=False)
                
                db = server.connect(host="192.168.100.200", user="mensajero", passwd="password", db="db_trading")
                query = db.cursor()

                query.execute(f"INSERT INTO fundamentales (emisora, periodo, eeff) VALUES ('{emisora[0]}', '{periodo_texto}', '{carga_eeff}')")
                db.commit()
                db.close()
                
                print(f"[OK] {emisora[0]} {periodo_texto}")
                time.sleep(40)            
            else:
                continue                
    else:
        print(f"La emisora {emisora} no se encuentra en la lista de links disponibles")
print("-> La Base de Datos se encuentra actualizada")

La emisora ('ACTINVR', 'CORPORACION ACTINVER, S.A.B. DE C.V.') no se encuentra en la lista de links disponibles
La emisora ('AGRIEXP', 'AGRO INDUSTRIAL EXPORTADORA, S.A. DE C.V.') no se encuentra en la lista de links disponibles
La emisora ('ANB', 'ANHEUSER-BUSCH INBEV SA/NV') no se encuentra en la lista de links disponibles
La emisora ('BBAJIO', 'BANCO DEL BAJÍO, S.A., INSTITUCIÓN DE BANCA MÚLTIPLE') no se encuentra en la lista de links disponibles
La emisora ('BBVA', 'BANCO BILBAO VIZCAYA ARGENTARIA, S.A.') no se encuentra en la lista de links disponibles
La emisora ('BSMX', 'BANCO SANTANDER MEXICO, S.A., INSTITUCION DE BANCA MULTIPLE, GRUPO FINANCIERO SANTANDER') no se encuentra en la lista de links disponibles
La emisora ('CREAL', 'CREDITO REAL, S.A.B. DE C.V., SOFOM, E.R.') no se encuentra en la lista de links disponibles
La emisora ('EDOARDO', 'EDOARDOS MARTIN, S.A.B. DE C.V.') no se encuentra en la lista de links disponibles
La emisora ('ELEMENT', 'ELEMENTIA, S.A.B. DE C.V.') no